In [52]:
from datetime import datetime, timedelta
import os
from time_utils.gpst import dt2gpst, gpst2dt, gpst_week_number, gpst_week_day

import subprocess

from ftplib import FTP
from os.path import basename, dirname, join, exists
from os import makedirs

import matplotlib.pyplot as plt

import numpy
import pandas as pd
from numpy import diff
from pandas import DataFrame

In [53]:
from RINEX2Helper import format_filepath, ftp_download, fix_bad_zip_file, decompress
from rinex2 import parse_RINEX2_obs_file
from rinex_nav import parse_rinex_nav_file
from gps_orbit import compute_gps_orbital_parameters_from_ephemeris, compute_ecef_position_from_orbital_parameters
from gps_orbit import compute_gps_orbital_parameters_from_ephemeris, compute_ecef_position_from_orbital_parameters
from coordinate_conversion import ecef2sky

In [54]:
def download_and_decompress(station_id, dt, overwrite=False):
    ftp_host = 'cddis.gsfc.nasa.gov'
    filepath_template = 'pub/gps/data/daily/{yyyy}/{ddd}/{yy}o/{station_id}{ddd}0.{yy}o.Z'
    rinex_dir = '/Users/liuzijun 1/Projects/gnss-research/data/rinex/'
    url_filepath = format_filepath(filepath_template, dt, params={'station_id': station_id})
    filepath = os.path.join(rinex_dir, url_filepath)
    if not os.path.exists(os.path.dirname(filepath)):
        os.makedirs(os.path.dirname(filepath))
    if overwrite or (not os.path.exists(filepath) and not os.path.exists(filepath[:-2])):
        success = ftp_download(ftp_host, url_filepath, filepath)
    if overwrite or (os.path.exists(filepath) and not os.path.exists(filepath[:-2])):
        subprocess.call('uncompress -f ' + filepath.replace(' ', '\ '), shell=True)
    filepath = filepath[:-2]
    return filepath 

In [55]:
from scipy.constants import c
kappa_u = 40.308e16
fL1 = 1.57542e9
fL2 = 1.2276e9

In [62]:
def download_and_decompress_nav(dt, overwrite=False):
    ftp_host = 'cddis.gsfc.nasa.gov'
    filepath_template = 'pub/gps/data/daily/{yyyy}/{ddd}/{yy}n/chan{ddd}0.{yy}n.Z'
    rinex_dir = '/Users/liuzijun 1/Projects/gnss-research/data/rinex/'
    url_filepath = format_filepath(filepath_template, dt)
    filepath = os.path.join(rinex_dir, url_filepath)
    print (url_filepath)
    if not os.path.exists(os.path.dirname(filepath)):
        os.makedirs(os.path.dirname(filepath))
    if overwrite or (not os.path.exists(filepath) and not os.path.exists(filepath[:-2])):
        success = ftp_download(ftp_host, url_filepath, filepath)
    if overwrite or (os.path.exists(filepath) and not os.path.exists(filepath[:-2])):
        subprocess.call('uncompress -f ' + filepath.replace(' ', '\ '), shell=True)
    filepath = filepath[:-2]
    return filepath 


In [63]:
def process_day(dt, station_id, data):
    nav_filepath = download_and_decompress_nav(dt)
    headers, nav_data = parse_rinex_nav_file(nav_filepath)
    filepath = download_and_decompress(station_id, dt)
    header, observations = parse_RINEX2_obs_file(filepath)
    satellites = observations['satellites']
    time = observations['time']
    sat_ids = sorted(filter(lambda sat_id: sat_id[0] == 'G', satellites.keys()))
    print('', end='')
    for sat_id in sat_ids:
        print('\r' + sat_id, end='')
        
        header, observations = parse_RINEX2_obs_file(filepath)
        sat = observations['satellites'][sat_id]
        time = observations['time'][sat['index']]
        rx_ecf = list(map(float, header['approximate_position_xyz'].split()))
        
        sat_prn = int(sat_id[1:])
        orbit_params = compute_gps_orbital_parameters_from_ephemeris(nav_data[sat_prn][0], time)
        sat_ecf = compute_ecef_position_from_orbital_parameters(orbit_params)
        sat_sky = ecef2sky(rx_ecf, sat_ecf)
        
        P1 = sat['L1']['pseudorange']
        P2 = sat['L2']['pseudorange']
        TEC = (P1 - P2) / (kappa_u * (1 / fL1**2 - 1 / fL2**2))

        L1 = sat['L1']['carrier'] * c / fL1
        L2 = sat['L2']['carrier'] * c / fL2
        TEC_rel = -(L1 - L2) / (kappa_u * (1 / fL1**2 - 1 / fL2**2))
    
        TEC_diff = diff(TEC_rel)
        threshold = 0.4
        jump_indices = numpy.where(abs(TEC_diff) > threshold)[0]
        
        date_str = '{0:04}{1:02}{2:02}'.format(dt.year, dt.month, dt.day)
        
        for jump_index in jump_indices:
            data['Date_str'].append(date_str)
            data['Satellite_id'].append(sat_id)
            data['Station_id'].append(station_id)
            data['Azimuth'].append(sat_sky[jump_index, 0])
            data['Elevation'].append(sat_sky[jump_index, 1])
            data['TEC_diff'].append(abs(TEC_diff)[jump_index])
    return filepath

In [64]:
data = {
    'Date_str': [],
    'Station_id': [],
    'Satellite_id': [],
    'Azimuth': [],
    'Elevation': [],
    'TEC_diff': []
}
station_id = 'chan'
year = 2017
dt_start = datetime(year, 1, 1)
filepaths = []

for day in range(365):
    
    
    dt = dt_start + timedelta(days=day)
    print(dt)
    try:
        process_day(dt, station_id, data)
    except Exception as e:
        print(e)
    print('')
    
    # delete extra files
    if day % 30 == 0:
        try:
            for filepath in filepaths:
                os.remove(filepath)
        except Exception as e:
            pass
        filepaths = []

2017-01-01 00:00:00
pub/gps/data/daily/2017/001/17n/chan0010.17n.Z
G01

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in greater


G32
2017-01-02 00:00:00
pub/gps/data/daily/2017/002/17n/chan0020.17n.Z
G32
2017-01-03 00:00:00
pub/gps/data/daily/2017/003/17n/chan0030.17n.Z
G32
2017-01-04 00:00:00
pub/gps/data/daily/2017/004/17n/chan0040.17n.Z
G32
2017-01-05 00:00:00
pub/gps/data/daily/2017/005/17n/chan0050.17n.Z
G32
2017-01-06 00:00:00
pub/gps/data/daily/2017/006/17n/chan0060.17n.Z
G32
2017-01-07 00:00:00
pub/gps/data/daily/2017/007/17n/chan0070.17n.Z
G32
2017-01-08 00:00:00
pub/gps/data/daily/2017/008/17n/chan0080.17n.Z
G32
2017-01-09 00:00:00
pub/gps/data/daily/2017/009/17n/chan0090.17n.Z
G32
2017-01-10 00:00:00
pub/gps/data/daily/2017/010/17n/chan0100.17n.Z
G32
2017-01-11 00:00:00
pub/gps/data/daily/2017/011/17n/chan0110.17n.Z
G32
2017-01-12 00:00:00
pub/gps/data/daily/2017/012/17n/chan0120.17n.Z
G32
2017-01-13 00:00:00
pub/gps/data/daily/2017/013/17n/chan0130.17n.Z
G32
2017-01-14 00:00:00
pub/gps/data/daily/2017/014/17n/chan0140.17n.Z
G32
2017-01-15 00:00:00
pub/gps/data/daily/2017/015/17n/chan0150.17n.Z
G32
20

G32
2017-04-28 00:00:00
pub/gps/data/daily/2017/118/17n/chan1180.17n.Z
G32
2017-04-29 00:00:00
pub/gps/data/daily/2017/119/17n/chan1190.17n.Z
G32
2017-04-30 00:00:00
pub/gps/data/daily/2017/120/17n/chan1200.17n.Z
G32
2017-05-01 00:00:00
pub/gps/data/daily/2017/121/17n/chan1210.17n.Z
G32
2017-05-02 00:00:00
pub/gps/data/daily/2017/122/17n/chan1220.17n.Z
G32
2017-05-03 00:00:00
pub/gps/data/daily/2017/123/17n/chan1230.17n.Z
G32
2017-05-04 00:00:00
pub/gps/data/daily/2017/124/17n/chan1240.17n.Z
G32
2017-05-05 00:00:00
pub/gps/data/daily/2017/125/17n/chan1250.17n.Z
G32
2017-05-06 00:00:00
pub/gps/data/daily/2017/126/17n/chan1260.17n.Z
G32
2017-05-07 00:00:00
pub/gps/data/daily/2017/127/17n/chan1270.17n.Z
G32
2017-05-08 00:00:00
pub/gps/data/daily/2017/128/17n/chan1280.17n.Z
G32
2017-05-09 00:00:00
pub/gps/data/daily/2017/129/17n/chan1290.17n.Z
G32
2017-05-10 00:00:00
pub/gps/data/daily/2017/130/17n/chan1300.17n.Z
G32
2017-05-11 00:00:00
pub/gps/data/daily/2017/131/17n/chan1310.17n.Z
G32
20

G32
2017-08-22 00:00:00
pub/gps/data/daily/2017/234/17n/chan2340.17n.Z
G32
2017-08-23 00:00:00
pub/gps/data/daily/2017/235/17n/chan2350.17n.Z
550 Failed to open file.

2017-08-24 00:00:00
pub/gps/data/daily/2017/236/17n/chan2360.17n.Z
G32
2017-08-25 00:00:00
pub/gps/data/daily/2017/237/17n/chan2370.17n.Z
G32
2017-08-26 00:00:00
pub/gps/data/daily/2017/238/17n/chan2380.17n.Z
G32
2017-08-27 00:00:00
pub/gps/data/daily/2017/239/17n/chan2390.17n.Z
G32
2017-08-28 00:00:00
pub/gps/data/daily/2017/240/17n/chan2400.17n.Z
G32
2017-08-29 00:00:00
pub/gps/data/daily/2017/241/17n/chan2410.17n.Z
G32
2017-08-30 00:00:00
pub/gps/data/daily/2017/242/17n/chan2420.17n.Z
G32
2017-08-31 00:00:00
pub/gps/data/daily/2017/243/17n/chan2430.17n.Z
G32
2017-09-01 00:00:00
pub/gps/data/daily/2017/244/17n/chan2440.17n.Z
G32
2017-09-02 00:00:00
pub/gps/data/daily/2017/245/17n/chan2450.17n.Z
G32
2017-09-03 00:00:00
pub/gps/data/daily/2017/246/17n/chan2460.17n.Z
G32
2017-09-04 00:00:00
pub/gps/data/daily/2017/247/17n

G32
2017-12-16 00:00:00
pub/gps/data/daily/2017/350/17n/chan3500.17n.Z
G32
2017-12-17 00:00:00
pub/gps/data/daily/2017/351/17n/chan3510.17n.Z
G32
2017-12-18 00:00:00
pub/gps/data/daily/2017/352/17n/chan3520.17n.Z
G32
2017-12-19 00:00:00
pub/gps/data/daily/2017/353/17n/chan3530.17n.Z
G32
2017-12-20 00:00:00
pub/gps/data/daily/2017/354/17n/chan3540.17n.Z
G32
2017-12-21 00:00:00
pub/gps/data/daily/2017/355/17n/chan3550.17n.Z
G32
2017-12-22 00:00:00
pub/gps/data/daily/2017/356/17n/chan3560.17n.Z
G32
2017-12-23 00:00:00
pub/gps/data/daily/2017/357/17n/chan3570.17n.Z
G32
2017-12-24 00:00:00
pub/gps/data/daily/2017/358/17n/chan3580.17n.Z
G32
2017-12-25 00:00:00
pub/gps/data/daily/2017/359/17n/chan3590.17n.Z
G32
2017-12-26 00:00:00
pub/gps/data/daily/2017/360/17n/chan3600.17n.Z
G32
2017-12-27 00:00:00
pub/gps/data/daily/2017/361/17n/chan3610.17n.Z
G32
2017-12-28 00:00:00
pub/gps/data/daily/2017/362/17n/chan3620.17n.Z
G32
2017-12-29 00:00:00
pub/gps/data/daily/2017/363/17n/chan3630.17n.Z
G32
20

In [66]:
df = DataFrame(data)
with open('Table_{station_id}_2017_all.csv'.format(station_id=station_id), 'w') as f:
    df.to_csv(f, index = None, header = True)

In [ ]:
# Introduction

- Global Positioning System (GPS) is an important technology
  - used for navigation and timing
  - also used for studying geophysical phenomena (earthquakes, ionosphere, surface reflections)
- Cycle slips are a disruption in GPS phase measurements
  - caused by low signal-to-noise ratio and/or interference
- In this work, we looked at the occurrence of cycle slips in GPS data from various sites around the globe
- Looking at jumps in ionosphere total electron content (TEC) can be a simple way to detect cycle slips
- We used the dual-frequency relative TEC measurements to detect cycle slips
- We then look at how the occurrence of cycle slips depends on satellite azimuth/elevation, site location, and time of day
- This type of analysis is helpful in determining where to focus efforts in mitigation of cycle slips in GPS data

In [17]:
# dfNew = pd.read_csv("Table_nist_2017_all.csv")
# SatID = ["G01", "G02", "G03", "G04", "G05", "G06", "G07", "G08", "G09", "G10", "G11", "G12", "G13", "G14", "G15", "G16", "G17", "G18", "G19", "G20", "G21", "G22", "G23", "G24", "G25", "G26", "G27", "G28", "G29", "G30", "G31", "G32"]
# for i in SatID:
#     dfSID = 'df_{}'.format(i)
#     dfSID = dfNew.loc[dfNew["Satellite_id"] == i]
#     dfSID.to_csv('Table_Azimuth_Elevation_{0}.csv'.format(i), index = None, header = True)

In [29]:
# data = {
#     'Date_str': [],
#     'Station_id': [],
#     'Satellite_id': [],
#     'Azimuth': [],
#     'Elevation': []
# }
# station_id = 'yell'

# for day in range(1, 32):
#     dt = datetime(2017, 1, day)
#     print(dt)
#     nav_filepath = download_and_decompress_nav(dt)
#     headers, nav_data = parse_rinex_nav_file(nav_filepath)
#     filepath = download_and_decompress(station_id, dt)
#     header, observations = parse_RINEX2_obs_file(filepath)
#     satellites = observations['satellites']
#     time = observations['time']
#     sat_ids = sorted(filter(lambda sat_id: sat_id[0] == 'G', satellites.keys()))
#     print('', end='')
#     for sat_id in sat_ids:
#         print('\r' + sat_id, end='')
        
#         header, observations = parse_RINEX2_obs_file(filepath)
#         sat = observations['satellites'][sat_id]
#         time = observations['time'][sat['index']]
#         rx_ecf = list(map(float, header['approximate_position_xyz'].split()))
        
#         sat_prn = int(sat_id[1:])
#         orbit_params = compute_gps_orbital_parameters_from_ephemeris(nav_data[sat_prn][0], time)
#         sat_ecf = compute_ecef_position_from_orbital_parameters(orbit_params)
#         sat_sky = ecef2sky(rx_ecf, sat_ecf)
        
#         P1 = sat['L1']['pseudorange']
#         P2 = sat['L2']['pseudorange']
#         TEC = (P1 - P2) / (kappa_u * (1 / fL1**2 - 1 / fL2**2))

#         L1 = sat['L1']['carrier'] * c / fL1
#         L2 = sat['L2']['carrier'] * c / fL2
#         TEC_rel = -(L1 - L2) / (kappa_u * (1 / fL1**2 - 1 / fL2**2))
    
#         TEC_diff = diff(TEC_rel)
#         threshold = 0.7
#         jump_indices = numpy.where(abs(TEC_diff) > threshold)[0]
        
#         date_str = '{0:04}{1:02}{2:02}'.format(dt.year, dt.month, dt.day)
        
#         for jump_index in jump_indices:
#             data['Date_str'].append(date_str)
#             data['Satellite_id'].append(sat_id)
#             data['Station_id'].append(station_id)
#             data['Azimuth'].append(sat_sky[jump_index, 0])
#             data['Elevation'].append(sat_sky[jump_index, 1])

#     print('')